# <center>DATA Cleaning</center>

## <center><u>``Resumo do código``</u></center>

>### Entra: 1Ninjas e 2Delta  
>### Sai: 1Ninjas_Limpo e 2Delta_Limpo

---
O objectivo é receber dados dos Ninjas e da Delta e devolver ficheiros prontos para ser trabalhados. 

Devolve os ficheiros que entram nos códigos 1 e 2.

---
- Inputs

> __Dados dos ninjas__ em formato Wide
> - 1's e 0's consoante a presença e ausência do produto, Data e Loja

> __Dados da Delta__ em formato Wide
> - Stocks e trânsito, Sellout dia anterior

- Outputs

> **Dados dos Ninjas** limpos

> **Dados da Delta** limpos

In [1]:
import pandas as pd
import numpy as np
import csv

def escrever_excel(dfa, nome):
    dfa.to_excel('%s.xlsx' %nome, index=False)
    
def escrever_csv(dfa, nome):
    dfa.to_csv(nome+'.csv', index=False)
    

In [2]:
1-9/77

0.8831168831168831

# Ler e Mudar nomes

- # `Ninjas`

In [100]:
#Info Ninja
dfNinjas=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão_05_2023\\1Ninjas.xlsx")

#Renomear colunas
dfNinjas = dfNinjas.rename(columns={' Data de Resposta': 'DATA', "Código da loja":"STORE", "Nome da Loja":"Loja"})

#Passar para datetime
dfNinjas['DATA'] = pd.to_datetime(dfNinjas['DATA'])

- Lista de lojas

In [101]:
ficheiroLojas = pd.DataFrame(dfNinjas["STORE"].unique().tolist())
#ficheiroProdutos = pd.DataFrame(dfNinjas["DESC_ARTIGO"].unique().tolist())

- #  `Delta (stocks, reposição e sellouts)`

In [103]:
%%time
# Ler o ficheiro long com Stocks e Fornecimento
dfDelta1=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão_05_2023\\2_Delta1.xlsx")
dfDelta2=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão_05_2023\\2_Delta2.xlsx")
dfDelta3=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão_05_2023\\2_Delta3.xlsx")
dfDelta4=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão_05_2023\\2_Delta4.xlsx")

dataframes=[dfDelta1,dfDelta2,dfDelta3,dfDelta4]
dfDelta=pd.concat(dataframes, ignore_index=True)

# Renomear
dfDelta['DATA'] = pd.to_datetime(dfDelta['DATA'], format='%d-%m-%Y')

CPU times: total: 25.4 s
Wall time: 35.3 s


> - **Definir os produtos e lojas em causa**

In [104]:
# Ler ficheiros com os nomes de produtos e números de loja
df_produtos = pd.read_csv('D:\\B&N Dados\\Delta\\Padrão_05_2023\\produtos.txt', header=None)
df_lojas = pd.read_csv('D:\\B&N Dados\\Delta\\Padrão_05_2023\\lojas.csv', header=None) 

# Passar para uma lista
produtos = df_produtos[0].tolist()
lojas = df_lojas[0].tolist()

# Estabelecer o Dataframe mais limitado
dfDelta_Limpo = dfDelta[dfDelta['STORE'].isin(lojas) & dfDelta['DESC_ARTIGO'].isin(produtos)].copy()

> - **Limitar os dados apresentados e Renomear colunas**

In [107]:
# Tirar apenas as colunas de interesse 
dfDelta_Limpo = dfDelta.loc[:, ['DATA', 'EAN', 'DESC_ARTIGO', 'STORE', "STORE_NAME", 
                         "SOH", "INTRANSIT", "EXPECTED", "PRES_STOCK", "VND (D-1)"]].copy()

# Mudar nomes
dfDelta_Limpo = dfDelta_Limpo.rename(columns={"VND (D-1)": "SELLOUT_1_Dias_Antes", "SOH": "STOCK_1_Dias_Antes"})

> - **Criar colunas de Stock e Sellout para o próprio dia**

In [108]:
# Organizar para atingir os valores relevantes
dfDelta_Limpo = dfDelta_Limpo.sort_values(by=["STORE", "DESC_ARTIGO", "DATA"])

#Agrupar por loja e produto para não haver valores errados 
dfDelta_Limpo["SELLOUT"] = dfDelta_Limpo.groupby(["DESC_ARTIGO","STORE"])["SELLOUT_1_Dias_Antes"].shift(-1)
dfDelta_Limpo["STOCK"] = dfDelta_Limpo.groupby(["DESC_ARTIGO","STORE"])["STOCK_1_Dias_Antes"].shift(-1)

> - **Retirar todos os dias que não têm interesse**

In [110]:
# O dia 29 foi utilizado para conseguir dados do dia para stocks e sellouts mas não é dia de missão
dfDelta_Limpo = dfDelta_Limpo.loc[dfDelta_Limpo['DATA'] <'2023-05-29'].copy()

In [47]:
df_Fundido=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão_05_2023\\Fundido.xlsx")

In [46]:
# Criar coluna com o número dos produtos em causa presentes no instante

df_Fundido["Num_Produtos"] = df_Fundido[produtos].sum(axis=1)

df_Fundido.loc[(df_Fundido["Num_Produtos"] == 0) & (df_Fundido[produtos[1]].isna()), "Num_Produtos"] = np.nan

In [39]:
df_Mergido= pd.merge(dfDelta1,dfFalta, how="left", on=["STORE"])

In [49]:
escrever_excel(df_Fundido, "Fundido")

In [27]:
# Ler ficheiro para dataframe
df_produtos = pd.read_csv('D:\\B&N Dados\\Delta\\Padrão_05_2023\\produtos.txt', header=None)

# Passar para uma lista
produtos = df_produtos[0].tolist()

In [48]:
#df_Fundido["Sinal"]= pd.Series(dtype='int')

# Define the mappings of numbers to strings
mapping = {1: "Presente c/ Stock",
           2: "Ausente c/ Stock",
           3: "Presente s/ Stock",
           4: "Ausente s/ Stock"}

for i in produtos:
    sinal = "SINAL %s" % i
    stock  = "STOCK %s" % i
    
    df_Fundido[sinal] = np.where((df_Fundido[stock] > 0) & (df_Fundido[i] == 1), 1,
                     np.where((df_Fundido[stock] > 0) & (df_Fundido[i] == 0), 2,
                     np.where((df_Fundido[stock] <= 0) & (df_Fundido[i] == 1), 3,
                     np.where((df_Fundido[stock] <= 0) & (df_Fundido[i] == 0), 4, 
                     np.nan))))


    # Map the numbers to the corresponding strings
    df_Fundido[sinal] = df_Fundido[sinal].map(mapping)
    

# 1: Há Stock e Sinal Positivo Ninja "Verdadeiramente Correcto"
# 2: Há Stock e Sinal Negativo Ninja "Falso Positivo"
# 3: Não há Stock e Sinal Positivo Ninja "Falso Negativo"
# 4: Não há Stock e Sinal Negativo Ninja "Rejeição Correcta"

In [33]:
df_columns = dfDelta1.columns.tolist()
indexed_columns = [f"{index}: {column}" for index, column in enumerate(df_columns)]
indexed_columns

['0: Vendedor',
 '1: Hora',
 '2: DATA',
 '3: Loja',
 '4: STORE',
 '5: CAFÉ DELTA Q QALIDUS 10CAP',
 '6: CAFÉ DELTA Q MYTHIQ 10CAP',
 '7: CAFÉ DELTA Q QHARACTER 10CAP',
 '8: CAFÉ GINGA 10CAP',
 '9: CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G',
 '10: CAFÉ DELTA RITUAL MU 220G',
 '11: CAFÉ TORRADO RITUAL DELTA GRÃO  500G',
 '12: CAFÉ DELTA MOAGEM UNIVERSAL BRASIL 220G',
 '13: CAFÉ DELTA MOAGEM UNIVERSAL ANGOLA 220G',
 '14: CAFÉ SOLÚVEL DELTA FRASCO 200G',
 '15: BEBIDA CEREAIS DELTA C/20%CAFE FR 200G',
 '16: Num_Produtos',
 '17: Reposição',
 '18: STOCK CAFÉ DELTA Q QALIDUS 10CAP',
 '19: PRESLINEAR CAFÉ DELTA Q QALIDUS 10CAP',
 '20: INTRANSIT CAFÉ DELTA Q QALIDUS 10CAP',
 '21: EXPECTED CAFÉ DELTA Q QALIDUS 10CAP',
 '22: SELLOUT CAFÉ DELTA Q QALIDUS 10CAP',
 '23: STOCK CAFÉ DELTA Q MYTHIQ 10CAP',
 '24: PRESLINEAR CAFÉ DELTA Q MYTHIQ 10CAP',
 '25: INTRANSIT CAFÉ DELTA Q MYTHIQ 10CAP',
 '26: EXPECTED CAFÉ DELTA Q MYTHIQ 10CAP',
 '27: SELLOUT CAFÉ DELTA Q MYTHIQ 10CAP',
 '28: STOCK CAFÉ DELTA Q QHARA

In [11]:
column_to_move = dfDelta1.iloc[:, 17]  # Extract the column at index 17 (Python is 0-indexed)

# Remove the column from its current position
dfDelta1.drop(dfDelta1.columns[17], axis=1, inplace=True)

# Shift the remaining columns to the right by one position
dfDelta1 = pd.concat([column_to_move, dfDelta1], axis=1)

# Escrever

In [111]:
escrever_excel(dfNinjas, "1Ninjas_Limpo")
escrever_excel(dfDelta_Limpo, "2Delta_Limpo")

#escrever_csv(ficheiroLojas, "lojas")
#escrever_csv(ficheiroProdutos, "produtos")